In [4]:
from config.config import BOOK_API_KEY
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd

In [5]:
api_service_name = "books"
api_version = "v1"

book_ids = [
    "JfLsAQAAQBAJ",  # Replace with actual book IDs
    # Can add more book IDs
]

# Get credentials and create an API client

books = build(api_service_name, api_version, developerKey=BOOK_API_KEY)

request = books.volumes().list(
    q=",".join(book_ids)
)
response = request.execute()

print(response)


{'kind': 'books#volumes', 'totalItems': 1, 'items': [{'kind': 'books#volume', 'id': 'JfLsAQAAQBAJ', 'etag': '2r7s8wpO664', 'selfLink': 'https://books.googleapis.com/books/v1/volumes/JfLsAQAAQBAJ', 'volumeInfo': {'title': 'Hamlet', 'authors': ['Shakespeare, William'], 'publisher': 'Aegitas', 'publishedDate': '2015-04-24', 'description': 'Hamlet é uma tragédia de William Shakespeare, escrita entre 1599 e 1601. A peça, situada na Dinamarca, reconta a história de como o Príncipe Hamlet tenta vingar a morte de seu pai Hamlet, o rei, executando seu tio Cláudio, que o envenenou e em seguida tomou o trono casando-se com a mãe de Hamlet. A peça traça um mapa do curso de vida na loucura real e na loucura fingida — do sofrimento opressivo à raiva fervorosa — e explora temas como a traição, vingança, incesto, corrupção e moralidade. Apesar da enorme investigação que se faz acerca do texto, o ano exato em que Shakespeare escreveu-o permanece em debate. Três primeiras versões da peça sobrevivem aos 

In [3]:
JSON(response)

<IPython.core.display.JSON object>

In [6]:
api_service_name = "books"
api_version = "v1"

book_ids = [
    "JfLsAQAAQBAJ",
    "ksOEEAAAQBAJ",
    "LPKyUG4OmlsC",
    "vAsdtG_P21oC"
]

# Get credentials and create an API client

books = build(api_service_name, api_version, developerKey=BOOK_API_KEY)

In [7]:
def get_book_info(books, book_ids):
    all_data = []

    """request = books.volumes().list(
        q=",".join(book_ids)
    )

    response = request.execute()

    # loop through items in response
    for item in response.get("items", []):
        volume_info = item.get("volumeInfo", {})"""

    for book_id in book_ids:
        request = books.volumes().get(volumeId=book_id)
        response = request.execute()

        volume_info = response.get("volumeInfo", {})

        data = {
            'title': volume_info.get('title', 'N/A'),
            'authors': volume_info.get('authors', 'N/A'),
            'publishedDate': volume_info.get('publishedDate', 'N/A'),
            'description': volume_info.get('description', 'N/A'),
            'pageCount': volume_info.get('pageCount', 'N/A'),
            'categories': volume_info.get('categories', 'N/A'),
        }
        all_data.append(data)

    return pd.DataFrame(all_data)


In [8]:
book_data = get_book_info(books, book_ids)

In [7]:
book_data

,title,authors,publishedDate,description,pageCount,categories
0,Hamlet,"[Shakespeare, William]",2015-04-24,"Hamlet é uma tragédia de William Shakespeare, ...",142,"[Drama / General, Drama / Shakespeare, Drama /..."
1,Macbeth - NE,[William Shakespeare],2012-01-01,O apaixonante deste texto de Shakespeare é o c...,114,[Drama / General]
2,Odisseia,[Homero],2007-04-25,"Surgida no século IX a.C., a ""Odisseia"" é cons...",456,[Fiction / Classics]
3,Noite de Reis,[William Shakespeare],2011-11-09,"Nesta peça, um grupo arma uma cilada para faze...",144,[Fiction / General]


In [9]:
def get_popular_books_by_category(books, category):
    # Fonksiyon, belirli bir kategorideki popüler kitapları almak için kullanılır.
    all_data = []

    # Google Books API'ye bir sorgu yapılıyor.
    request = books.volumes().list(
        q=f"subject:{category}",  # Belirli bir kategoriye ait kitapları sorgulamak için kullanılan sorgu ifadesi.
        orderBy="relevance",  # Kitapların sıralama ölçütü. "relevance" en ilgili olanları ön plana çıkarır.
        maxResults=40  # En fazla alınacak kitap sayısı.
    )

    # API'ye yapılan sorgu sonucunda gelen yanıt alınıyor.
    response = request.execute()

    # Gelen yanıt içindeki her bir kitap için bir döngü oluşturuluyor.
    for item in response.get("items", []):
        # Kitap bilgilerini içeren kısmı alıyoruz.
        volume_info = item.get("volumeInfo", {})

        # Kitap bilgileri bir sözlük içinde düzenleniyor.
        data = {
            'title': volume_info.get('title', 'N/A'),
            'authors': volume_info.get('authors', ['N/A']),
            'publishedDate': volume_info.get('publishedDate', 'N/A'),
            'description': volume_info.get('description', 'N/A'),
            'pageCount': volume_info.get('pageCount', 'N/A'),
            'categories': volume_info.get('categories', ['N/A']),
        }
        
        # Kitap bilgileri listeye ekleniyor.
        all_data.append(data)

    # Oluşturulan kitap bilgileri listesi bir Pandas DataFrame'e dönüştürülüyor ve geri döndürülüyor.
    return pd.DataFrame(all_data)

##### Aşağıda categories kısmında "Literary Criticism" gibi Drama'dan farklı kategoriler gözükmektedir bunun sebebi Google Books API, bir kitabın kategorilerini belirlemek için çeşitli kaynakları kullanır ancak kitaplar genellikle birkaç farklı kategoriyle etiketlenebilir ve bu etiketler kitabın içeriğine, konusuna veya bir dizi diğer faktöre dayanabilir bu nedenle API tarafından döndürülen bir kitabın kategorileri sadece belirli bir anahtar kelimeyle eşleşmeyebilir.

In [11]:
popular = get_popular_books_by_category(books, category="Data Science")
popular

,title,authors,publishedDate,description,pageCount,categories
0,Provenance and Annotation of Data,[Luc Moreau],2006-09-27,This book constitutes the thoroughly referred ...,298,[Business & Economics]
1,Provenance and Annotation of Data,[Luc Moreau],2006-09-27,This book constitutes the thoroughly referred ...,298,[Business & Economics]
2,Proceedings of the 2nd International Conferenc...,"[Association for Computing Machinery, ACM Digi...",2009-11-24,ICIS 2009: The 2nd International Conference on...,1479,[Computer security]
3,Fallout,[Roger Cross],2001-01,"In the 1950s, the Australian government agreed...",226,[History]
4,IERI Monograph Series on Issues and Methodolog...,[N/A],2009,N/A,N/A,[N/A]
5,Education and Informatics Worldwide,[Jacques Hebenstreit],1992,The micro computer has had a revolutionary imp...,264,[Computer]
6,Education and Informatics Worldwide,[Jacques Hebenstreit],1992,The micro computer has had a revolutionary imp...,264,[Computer]
7,Underlying Dimensions of Knowledge Assessment,"[Derek Hung Chiat Chen, Kishore S. Gawande]",2007,The Knowledge Assessment Methodology (KAM) dat...,56,[Correlation]
8,The UK's Share of World Research Output,[Research Information Network (Organization)],2009,N/A,23,[Bibliometrics]
9,IPC for Modular Software Requires a Third Part...,"[Stuart Arthur Friedberg, ROCHESTER UNIV NY DE...",1987,"Reconfiguration, on-line maintenance, load bal...",11,[Data]


In [10]:
def get_book_info_and_url(books, query):
    request = books.volumes().list(q=query)
    response = request.execute()

    if response.get("items"):
        book_info = response["items"][0]["volumeInfo"]
        title = book_info.get("title", "")
        authors = ", ".join(book_info.get("authors", []))

        # Oluşturulan URL
        google_books_url = f"https://books.google.com.tr/books?id={response['items'][0]['id']}"

        return {
            'title': title,
            'authors': authors,
            'google_books_url': google_books_url
        }
    else:
        return None

In [11]:
info= get_book_info_and_url(books, query="Harry Potter")
info

{'title': 'Harry Potter ve Sirlar Odasi',
 'authors': 'J.K. Rowling',
 'google_books_url': 'https://books.google.com.tr/books?id=Dg5MEAAAQBAJ'}